# Exercise 3

In this assignment, you will find numerical solutions to the diffusion equation. In particular, you will use an implicit method, and consider problems with both Dirichlet and Neumann boundary conditions.

**Remember**
   * You are expected to use numpy and scipy libraries where appropriate.  
   * You should run each cell in order from the top of the notebook; there is no need to repeat code between cells
   * Use the "refresh kernel" button to reset everything and start again
   * Make sure your notebook runs fully & without errors, from a fresh kernel, before submitting it

## Problem Overview

The 1D diffusion equation is :

$$\frac{\partial u}{\partial t} = k\frac{\partial^2 u}{\partial x^2}$$

You should discretize this equation onto $N_x$ space points, with separation $\Delta x = h$, and into timesteps $\Delta t = \tau$.  In the equations below, I use subscript $i$ as a space index, and superscript $n$ for time indices.

Having discretized the problem, you should use the _implicit_ finite difference equation, as discussed in lectures :

$$\frac{u_i^{n+1} - u_i^n}{\tau} = k \frac{u_{i+1}^{n+1} - 2u_i^{n+1} + u_{i-1}^{n+1}}{h^2}$$

This can be written in matrix form $u^n = M u^{n+1}$ using :

$$u_i^n = - \alpha u_{i-1}^{n+1} + (1 + 2\alpha) u_i^{n+1} - \alpha u_{i+1}^{n+1}$$

where $\alpha = \frac{k \tau}{h^2}$.

In the problems below, you are asked to solve the diffusion equation in the context of the heat equation. Here, $k$ is the thermal diffusivity, given by $k = \frac{\lambda}{\rho C}$, where $\lambda$ is the thermal conductivity, $\rho$ is the density, and $C$ is the specific heat capacity. The questions below concern an iron poker of length 50cm.  You may take the thermal conductivity of iron to be a constant 59 W/m/K, its specific heat as 450 J/kg/K, and its density as 7,900 kg/m3.  You can ignore heat loss along the length of the poker.


## Part 1 - Dirichlet Boundary Conditions

The poker is initially in equilibrium, at room temperature of 20 C. At time $t = 0$, one end is thrust into a furnace at 1000 C and the other end is held in an ice bath at 0 C. Your task is to calculate the temperature distribution along the poker as a function of time.

The fact that the ends of the rod are held at fixed temperatures of 0 C and 1000 C corresponds to a Dirichlet boundary condition.  These can be included in the implicit method as follows.

The implicit finite difference equation, above, will allow us to calculate the unknown 'internal' nodes, ie. $0 < i < (N_x-1)$.  However, the boundary nodes, $i=0, N_x$, must have fixed values $d_0, d_N$.  To fix the boundaries, we take the matrix M to be of size $(N_x-2) \times (N_x-2)$, and adding a vector term :

$$u^n = Mu^{n+1} + b$$

For $N_x = 7$ (for example), this gives :

$$M = \begin{pmatrix}
1+2\alpha & -\alpha   &           &           &           \\
-\alpha   & 1+2\alpha & -\alpha   &           &           \\
          & -\alpha   & 1+2\alpha & -\alpha   &           \\
          &           & -\alpha   & 1+2\alpha & -\alpha   \\
          &           &           & -\alpha   & 1+2\alpha 
\end{pmatrix}$$

$$b = \begin{pmatrix}
-\alpha d_0 \\
0 \\
0 \\
0 \\
-\alpha d_N \\
\end{pmatrix}$$

You can show this gives the required finite equation for $i=1, (N-1)$, eg. :

$$u^n_1 = - \alpha u^{n+1}_2 + (1 + 2\alpha)u^{n+1}_1 - \alpha d_0$$

First, write functions that will construct the matrix equation and boundary value term.

In [ ]:
import unittest
import numpy as np
from scipy.sparse.linalg import inv as invert_sparse

# Solution is a data-class used to organise the objects required to 
# solve the diffusion equation.


class Diffusion_Solution:
    
    """Data-class used to organise the objects required to implicitly 
    solve the diffusion equation
    
    Parameters
    ----------
    boundary_types: tuple
        The possible boundary types that may be used.
    type: int
        The index of 'boundary_types' corresponding to the type of 
        boundary conditions used.

    bounds: NOTE fill in later!!

    matrix: array-like
        Matrix used to solve the diffusion equation for the specific
        boundary conditions
    
    coords: NOTE fill in later!

    """
    boundary_types = ("Dirichelt", "Neumann")

    def __init__(self, type):
        self.type = self.boundary_types[type]
        self.bounds = None
        self.matrix = None
        self.coords = None



def create_dirichlet(x_n, x_step, t_step, k, bounds):
    """Creates matrix equation required to solve the diffusion equation
    in 1D using Dirichlet boundary conditions

    Parameters
    ----------
    x_n: int
        The number of spatial steps.
    x_step: float
        The size of each spatial step.
    t_step: float
        The size of each time step.
    k: float
        The thermal diffusivity.
    bounds: array-like
        The ordered boundary conditions at either end of the 1D shape. 
        The first boundary condition will correspond to the 0th element 
        of the final solution while the first element will correspond 
        to the final element. 

    Returns
    -------
    inv_matrix: ndarray
        A two dimensional array containing the inverse matrix diffusion
        matrix required to solve the diffusion equation in 1D.
    bounds: ndarray
        The Dirichlet bounds applied to a one dimensional array.        
    """
    # Create the marix M
    # Check that x_n is of the correct form
    if (n_elements := int(x_n)-2) < -1:
        raise ValueError("Problem must contain at least one spacial step")
    alpha = k*t_step/x_step^2
    matrix = np.identity(n_elements, dtype=np.float64) * (1-2*alpha)
    np.fill_diagonal(matrix[1:, :-1], alpha)
    np.fill_diagonal(matrix[:-1, :1], alpha)

    # Determine the inverse of M
    matrix_inv = invert_sparse(matrix)

    # Create the boundary vector
    bound_arr = np.zeros(n_elements)
    bound_arr[0], bound_arr[-1] = bounds
    bound_arr *= alpha
    return matrix_inv, bound_arr


    
    # Want to create a Diffusion_Solution class
    # Want to create and return the inverse matrix
        # require number of x coords
        # require k in diffusion equation
        # require size of time step
        # require size of space step
    # Want to create and return the boundary vector



# Write temperature-over-time data-class, this will contains the time 
# array and any other features that become nccessary later down the 
# line.

# Write function that creates and instance of the temp-over-time data-
# class and populates it with the required matrix and array.


class Test_Create_Dirichlet(unittest.TestCase):
    """Testing 'create_dirichlet' function.
    """
    def setUp(self):
        """Initialize the required parameters to print the test results
        """
        self.test_name = None

    def tearDown(self):
        """Print the test results to std.out
        """
        print(f"{self.test_name}:")

    def test_bad_input(self):
        """Test that the function responds correctly to incorrect 
        inputs
        """
        self.test_name = "TEST BAD INPUT"


Now write a function which will transport the temperature distribution at time step $n$ to time step $n+1$. You will need to use an appropriate linear algebra routine.

Finally, use the functions above to calculate the temperature distribution as a function of time, and display this graphically using an appropriate plotting routine.

In [ ]:
# In this section make a recursive argorithm that runs until the it 
# either reaches the required bounds or is interrupted by the user. 
# This will require a second thread to run with a prompt to allow the
# user to quit if required. 

## Part 2 - Neumann Boundary Conditions

Now we assume the far end of the poker from the furnace is no longer held at 0 C, but instead experiences no heat loss. Again your task is to find the tempeterature distribution as a function of time.

In this case, you will need to implement a Neumann boundary condition at the end of the poker, to ensure the derivative $\frac{\partial u}{\partial x}$ is zero. Since we are using finite differences, this is equivalent to ensuring the final two noces have the same value.

The finite difference equation for node $i=(N-1)$ is :

$$u^n_{N-1} = -\alpha u^{n+1}_{N-2} + (1 + 2\alpha)u^{n+1}_{N-1} - \alpha u^{n+1}_{N}$$

To enforce the Neumann boundary condition we can substitute $u^{n+1}_{N} = u^{n+1}_{N-1}$, giving :

$$u^n_{N-1} = -\alpha u^{n+1}_{N-2} + (1 + \alpha)u^{n+1}_{N-1}$$

This results in a modified form of $M$, shown here for the example $N_x=7$, and the matrix ix $5\times5$ :  

$$M = \begin{pmatrix}
1+2\alpha & -\alpha   &           &           &           & \\
-\alpha   & 1+2\alpha & -\alpha   &           &           & \\
          & -\alpha   & 1+2\alpha & -\alpha   &           & \\
          &           & -\alpha   & 1+2\alpha & -\alpha   & \\
          &           &           & -\alpha   & 1+\alpha & \\
\end{pmatrix}$$

Note that you will also need to include a boundary term vector $b$, since the end of the poker in the furnace still requires a Dirichlet condition.

First write any new functions you need. You should be able to re-use some functions from Part 1.

Finally, use the functions above to calculate the temperature distribution as a function of time, and display this graphically using a sensible plotting function.

# Part 3

In the Markdown cell below, describe how your code solves the problem set. State any equations used in the solution and how they were obtained. Include your reasons for any libraries you used, as well as any particular programming techniques. Explain your choice of any test cases. Finally, state any salient features of the results you obtained. You are not expected to write more than about 250-300 words.

*Taylor series expansion to approximate second derivative of function*

$$ f(a+x) = \sum_{i=0}^{\infty}{\frac{f^i(a)x^i}{i!}} \approx f(a) + xf'(a) + \frac{1}{2}x^2f''(a)$$
$$ f(a-x) = \sum_{i=0}^{\infty}{\frac{(-1)^if^i(a)x^i}{i!}} \approx f(a) -xf'(a) + \frac{1}{2}x^2f(a)$$
$$ \Rightarrow f(a+x) + f(a-x) \approx f(a) + xf'(a) + \frac{1}{2}x^2f''(a) + f(a) -xf'(a) + \frac{1}{2}x^2f(a)$$
$$ \Rightarrow f''(a) \approx f(a+x) + f(a-x) -2f(a)$$


Probably want to use successive over relaxation (SOR) method as this requires $ \sim \frac{1}{3}J $ iterations to converge compared to the $ \sim \frac{1}{2}J^2 $ interations required for the Jacobi and $ \sim \frac{1}{4} J^2 $ required for the Gauss-Seidel method.
The SOR method is given by the following equation(s):

$$ V'_{i,j} = V_{i,j} = \omega\left(\frac{V'_{i-1,j} + V_{i+1,j} + V'_{i,j-1}}{4} - (\mathrm{RH side}) - V_{i,j} \right) $$


Note that one requires $\omega \in (0, 2)$ for SOR to converge (see Numerical Recipes 3rd edition, chapter 20.5.1, page 1062)

*Crank-Nicholson method is a good way to model evolution in time*

Note: when using the implicit method one is evaluating $u_{t+1}$ through the relation:
$$ M^{-1}(u_{t+0} - b_t) = u_{t+1} $$